# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [2]:
function LoopFields(c)
    return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, P=big"0.100000000000000005551115"+big"0.100000000000000005551115"*im, diagonal=true)]
    )
end

LoopSpectrum(c, Δmax) = Spectrum(LoopFields(c), Δmax, interchiral=true);

function solve(;indices, signature, parity, even_spin=(), benchmark=false)
    ext_fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(ext_fields..., Δmax=Δmax)
    SOn = LoopSpectrum(c, Δmax)
    Seven = Spectrum([V for V in SOn.fields if spin(V) % 2 == 0], Δmax, interchiral=true)

    specs = (; (chan =>
        if chan in even_spin
            Seven
        else
            SOn
        end
                for chan in (:s, :t, :u)
    )...)

    chan_spectra = ChannelSpectra(
        co, specs, signature;
        interchiral=true, Δmax=Δmax, parity=1
    )

    sys = BootstrapSystem(chan_spectra)
    evaluate_blocks!(sys)
    compute_linear_system!(sys)
    solve!(sys)
    if benchmark
        println("precomputation of blocks coefficients")
        @btime ChannelSpectra(
            $co, $SOn, $signature;
            interchiral=true, Δmax=Δmax, parity=1,
            exclude=(u=[Field(c, r=2, s=0)],)
        )
        println("time to compute all positions' cache")
        @btime BootstrapSystem($chan_spectra)
        println("time to evaluate blocks")
        @btime evaluate_blocks!($sys)
        println("time for the linear solver")
        @btime solve!($sys)
    end
    return sys
end

solve (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [ ]:
Δmax, prec = 30, 25
setprecision(BigFloat, prec, base=10);
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
sig = (s=0, t=1, u=1)
sol = solve(indices=ind, signature=sig, parity=0, even_spin=(:s,), benchmark=false)
show(stdout, sol.structure_constants, rmax=2)

UndefVarError: UndefVarError: `Spectrum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
fields = [Field(c, r=r, s=s) for (r, s) in ind]
co = Correlation(fields..., Δmax=Δmax)
V = Field(c, r=1, s=1, diagonal=true)
println(Block(co, :s, V; interchiral=true, Δmax=Δmax)(0.3+0.1im))
V = Field(c, P=P_rs(1, 1+big"1e-20", c), diagonal=true)
println(Block(co, :s, V; interchiral=true, Δmax=Δmax)(0.3+0.1im))

1.03225943089923665861986887 + 0.374522569951509185448186576im
1.03225943089923665863311079 + 0.374522569951509185440983152im


Benchmarks:
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [24]:
c = CentralCharge(β=1/(big"0.8"+big"0.2"*im))
indices = ((1, 0), (1, 0), (1//2, 0), (1//2, 0))
Vs = [Field(c, r=r, s=s) for (r, s) in indices]
co = Correlation(Vs..., Δmax=Δmax)

NonChiralCorrelation{Complex{BigFloat},NTuple{4, Field{Complex{BigFloat}}}} with external fields
< (1, 0) (1, 0) (1//2, 0) (1//2, 0) >

In [23]:
Δmax

30

In [16]:
ind = ((1, 0), (1, 0), (1//2, 0), (1//2, 0))
sig = (s=1, t=1//2, u=3//2)
show(stdout, solve(indices=ind, signature=sig, parity=1).structure_constants, rmax=4)

Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        0.485267-0.172024im │    2.90024e-18 │
│    (2, 0) │     -0.0095726+0.0442801im │    9.35306e-17 │
│ (2, 1//2) │    -0.00338609+0.0250186im │    7.29583e-18 │
│    (2, 1) │                          0 │        1.54005 │
│    (3, 0) │   8.6067e-06+2.26471e-07im │    1.29366e-12 │
│ (3, 1//3) │  7.07313e-06-1.98136e-06im │    9.36089e-13 │
│ (3, 2//3) │  2.97012e-06-2.89236e-06im │    6.23749e-13 │
│    (3, 1) │                          0 │       0.495952 │
│    (4, 0) │  9.58058e-11-3.25449e-11im │     1.39489e-5 │
│ (4, 1//4) │  2.02896e-11+1.12297e-12im │     6.07341e-5 │
│ (4, 1//2) │ -4.27366e-13+7.53728e-12im │     0.00010652 │
│ (4, 3//4) │  3.71681e-11-2.87711e-11im │     7.02859e-6 │
│    (4, 1) │                 

In [17]:
sig = (s=1, t=1//2, u=3//2)
sol = solve(indices=ind, signature=sig, parity=1).structure_constants
show(stdout, sol, rmax=4)

Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        0.485267-0.172024im │    1.89635e-18 │
│    (2, 0) │     -0.0095726+0.0442801im │    4.12181e-17 │
│ (2, 1//2) │    -0.00338609+0.0250186im │    1.76386e-17 │
│    (2, 1) │                          0 │        13.3405 │
│    (3, 0) │   8.6067e-06+2.26471e-07im │    1.10631e-12 │
│ (3, 1//3) │  7.07313e-06-1.98136e-06im │    1.15802e-12 │
│ (3, 2//3) │  2.97012e-06-2.89236e-06im │    7.05397e-13 │
│    (3, 1) │                          0 │       0.804119 │
│    (4, 0) │  9.58043e-11-3.25471e-11im │      1.6137e-5 │
│ (4, 1//4) │   2.0291e-11+1.12485e-12im │     7.00796e-5 │
│ (4, 1//2) │ -4.28395e-13+7.53611e-12im │    0.000123996 │
│ (4, 3//4) │  3.71686e-11-2.87707e-11im │     8.24699e-6 │
│    (4, 1) │                 

In [7]:
sig = (s=1, t=3//2, u=1//2)
show(stdout, solve(indices=ind, signature=sig, parity=1).structure_constants, rmax=4)

Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │      -0.502183+0.0830317im │    1.44754e-19 │
│    (2, 0) │     0.00239155+0.0255139im │    1.27812e-17 │
│ (2, 1//2) │    -0.00273704-0.0200265im │    2.03042e-18 │
│    (2, 1) │                          0 │       0.357752 │
│    (3, 0) │ -6.40312e-07-1.92164e-06im │    1.14574e-12 │
│ (3, 1//3) │  7.07647e-07+9.43683e-07im │    1.43454e-12 │
│ (3, 2//3) │ -6.00257e-07+2.35486e-07im │    1.17577e-12 │
│    (3, 1) │                          0 │      0.0847972 │
│    (4, 0) │  1.40209e-12+1.38757e-12im │    0.000240422 │
│ (4, 1//4) │  3.64093e-13-6.92633e-13im │    0.000531346 │
│ (4, 1//2) │  -1.5808e-13+4.59269e-13im │    0.000554518 │
│ (4, 3//4) │  -1.52972e-12-4.1919e-13im │     6.94485e-5 │
│    (4, 1) │                 

# Exact checks

In [26]:
rho(r, s, r1, s1, r2, s2, beta) = (-1)^(s * min(r, abs(r1 - r2)) * (r1 < r2)) * prod(
    2 * cos(π * (j * beta^2 + (s - s1 - pm * s2)/2))
    for pm in (-1, 1)
    for j in (- (r-1 - abs(r1 + pm * r2))/2):1:((r-1 - abs(r1 + pm * r2))/2)
)

rho (generic function with 1 method)

In [27]:
rho(1, 1//2, 1, 1, 1, 1, c.β)

1.41421356237309504880168876 + 0.0im

# Torus one point functions

## r = 1

In [12]:
V1 = Field(c, r=1, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1//2, t=0, u=1//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (1//2, 0) │     -91064.2-1.00991e+06im │       0.754226 │
│    (3//2, 0) │ -1.16538e+09-5.36607e+08im │        1.06624 │
│ (3//2, 2//3) │ -1.17969e+09-4.80705e+08im │        1.01252 │
│    (5//2, 0) │ -1.33268e+13-1.75488e+13im │        1.04056 │
│ (5//2, 2//5) │ -1.35296e+13-1.44892e+13im │        1.05596 │
│ (5//2, 4//5) │ -1.34353e+13-1.61626e+13im │        1.07512 │
│    (7//2, 0) │ -7.86626e+19+6.72128e+18im │       0.349374 │
│ (7//2, 2//7) │    4.47455e+18-1.754e+19im │        1.32508 │
│ (7//2, 4//7) │ -6.19279e+19+2.50687e+18im │       0.279845 │
│ (7//2, 6//7) │ -2.49757e+19-8.98874e+18im │        1.72398 │
└──────────────┴────────────────────────────┴────────────────┘
Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure co

## r = 2

In [13]:
V1 = Field(c, r=2, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1//2, t=1//2, u=1)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=0
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌───────────────┬────────────────────────────┬────────────────┐
│ Field         │ Structure constant         │ Relative error │
├───────────────┼────────────────────────────┼────────────────┤
│     (1//2, 0) │                  1.0+0.0im │            0.0 │
│     (3//2, 0) │          2.95326+208.582im │      0.0958125 │
│  (3//2, 2//3) │          102.232-75.4409im │      0.0428432 │
│ (3//2, -2//3) │           -192.386-12.26im │      0.0504616 │
│     (5//2, 0) │  3.85212e+07+3.04205e+07im │      0.0313957 │
│ (5//2, -2//5) │  -5.09855e+07+3.1446e+06im │       0.037878 │
│  (5//2, 2//5) │ -1.32077e+07-4.19787e+07im │       0.109298 │
│ (5//2, -4//5) │  3.25644e+07-3.19904e+07im │       0.105568 │
│  (5//2, 4//5) │ -8.77304e+06+4.08498e+07im │       0.149723 │
│     (7//2, 0) │  3.08858e+15-1.14152e+16im │       0.475499 │
│ (7//2, -2//7) │   7.38679e+15+1.1468e+16im │       0.419316 │
│  (7//2, 2//7) │ -6.64808e+15+3.81514e+15im │       0.688425 │
│  (7//2, 4//7) │  2.40496e+15

## r = 3

In [21]:
V1 = Field(c, r=3, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1, t=1, u=1)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -1.20182-0.0279884im │    4.95476e-22 │
│    (2, 0) │     10.393-172.132im │    5.26264e-21 │
│ (2, 1//2) │    -11.8153+223.25im │    6.71368e-21 │
│    (2, 1) │    -8.24649-107.44im │    6.27627e-21 │
│    (3, 0) │   -2810.78-3963.38im │    4.62018e-16 │
│ (3, 1//3) │    1334.42+1842.49im │    1.06977e-15 │
│ (3, 2//3) │    1286.43+1719.91im │    9.92159e-16 │
│    (3, 1) │   -2517.53-3212.67im │    5.18203e-16 │
│    (4, 0) │   -19661.1-10007.8im │     1.09724e-7 │
│ (4, 1//4) │    -2778.97-1616.8im │     7.48827e-7 │
│ (4, 1//2) │    24918.5+12939.0im │      8.4217e-8 │
│ (4, 3//4) │   -2672.28-1510.67im │     7.48701e-7 │
│    (4, 1) │    -18140.5-8665.5im │     1.09689e-7 │
└───────────┴──────────────────────┴────────────────┘
Channel t
┌───────

In [18]:
V1 = Field(c, r=3, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1, t=1//2, u=3//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │       -0.66214+0.0378776im │        1.65443 │
│    (2, 0) │          2377.38+990.313im │        3.88001 │
│ (2, 1//2) │         -1986.84-1529.71im │        17.8384 │
│    (2, 1) │          2148.54+2551.04im │       0.673263 │
│    (3, 0) │  -5.3921e+08-2.21053e+09im │         1.1336 │
│ (3, 1//3) │  7.88527e+08+2.31665e+09im │        1.11928 │
│ (3, 2//3) │  -3.6758e+08-2.04648e+09im │        1.14887 │
│    (3, 1) │  5.60215e+08+2.12719e+09im │        1.15467 │
│    (4, 0) │    1.91263e+19+3.153e+18im │       0.323662 │
│ (4, 1//4) │ -1.90079e+19-3.12482e+18im │       0.323588 │
│ (4, 1//2) │  1.86819e+19+3.00204e+18im │       0.322871 │
│ (4, 3//4) │ -1.81441e+19-2.80683e+18im │       0.322432 │
│    (4, 1) │  1.74415e+19+2.5

In [19]:
V1 = Field(c, r=3, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=3//2, t=0, u=3//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌──────────────┬───────────────────────────┬────────────────┐
│ Field        │ Structure constant        │ Relative error │
├──────────────┼───────────────────────────┼────────────────┤
│    (3//2, 0) │ 2.03739e+08-6.49688e+08im │        1.62024 │
│ (3//2, 2//3) │ 9.70682e+07-4.15013e+08im │        1.62368 │
│    (5//2, 0) │  8.14701e+12-2.0176e+12im │        1.27275 │
│ (5//2, 2//5) │ 9.41316e+12-1.64923e+12im │        1.55008 │
│ (5//2, 4//5) │ 6.84567e+12-1.02544e+12im │        1.25208 │
│    (7//2, 0) │ 8.22549e+18-4.93683e+18im │       0.961498 │
│ (7//2, 2//7) │ 5.92746e+18-1.36622e+19im │        1.06291 │
│ (7//2, 4//7) │ 1.01417e+19-5.36784e+18im │       0.922857 │
│ (7//2, 6//7) │ 3.78692e+18-1.08331e+19im │        1.18819 │
└──────────────┴───────────────────────────┴────────────────┘
Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼───────────────────────

In [ ]:
import BootstrapVirasoro: qfromx, qfromτ, xfromq

c_s = CentralCharge(β = c.β / sqrt(big(2)))
V1_s = Field(c_s, r=3, s=0)
V012 = Field(c_s, r=0, s=1//2, diagonal=true)
co_s = Correlation(V012, V1_s, V012, V012, Δmax=Δmax)
SOn_s = LoopSpectrum(c_s, :On, co_s.Nmax)
sig_s = (s=2, t=2, u=2)
chan_spectra_s = ChannelSpectra(
    co_s, SOn_s, sig_s;
    interchiral=true, Δmax=Δmax, parity=1
)
sys_s = BootstrapSystem(chan_spectra_s)

evaluate_blocks!(sys_s)
compute_linear_system!(sys_s)
solve!(sys_s)
show(stdout, sys_s.structure_constants, rmax=4)

Channel s
┌────────────┬───────────────────────────┬────────────────┐
│ Field      │ Structure constant        │ Relative error │
├────────────┼───────────────────────────┼────────────────┤
│     (2, 0) │                 1.0+0.0im │            0.0 │
│  (2, 1//2) │      0.718414-0.0625572im │       0.135819 │
│ (2, -1//2) │    -0.0667113+0.0646718im │       0.135819 │
│     (2, 1) │       -1.39399-0.093471im │    6.16389e-14 │
│     (3, 0) │                         0 │        2.13997 │
│ (3, -1//3) │                         0 │        3.87727 │
│  (3, 1//3) │                         0 │        2.74034 │
│ (3, -2//3) │                         0 │        2.20956 │
│  (3, 2//3) │                         0 │        3.45305 │
│     (3, 1) │                         0 │        7.41995 │
│     (4, 0) │ 0.000106874-0.000867714im │     3.77791e-9 │
│  (4, 1//4) │                         0 │       0.135819 │
│ (4, -1//4) │                         0 │       0.135819 │
│ (4, -1//2) │ 9.20701e-05+0.0

In [130]:
Δmax, prec = 50, 42
setprecision(BigFloat, prec, base=10)
c_s = CentralCharge(β = c.β / sqrt(big(2)))
V1_s = Field(c_s, r=1, s=0)
V2_s = Field(c_s, P=0.8+0.3im, diagonal=true)
V3_s = Field(c_s, P=0.9+0.2im, diagonal=true)
V4_s = Field(c_s, P=0.4+0.3im, diagonal=true)

V012 = Field(c_s, P=0.5+0.4im, diagonal=true)
co_s = Correlation(V2_s, V3_s, V1_s, V1_s, Δmax=Δmax)
SOn_s = LoopSpectrum(c_s, :On, co_s.Nmax)
sig_s = (s=0, t=1, u=1)
chan_spectra_s = ChannelSpectra(
    co_s, SOn_s, sig_s;
    interchiral=true, Δmax=Δmax, parity=1
)
sys_s = BootstrapSystem(chan_spectra_s)

evaluate_blocks!(sys_s)
compute_linear_system!(sys_s)
solve!(sys_s)
show(stdout, sys_s.structure_constants, rmax=4)

Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.20+0.80im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │         -29.3399+52.7738im │        0.57875 │
│          (1, 1) │                          0 │       0.483225 │
│          (2, 0) │         -6.48271-10.1169im │       0.578563 │
│       (2, 1//2) │                          0 │         0.7867 │
│          (2, 1) │         -1.35258+12.6652im │       0.578232 │
│          (3, 0) │        -1.81889-0.346372im │       0.577983 │
│       (3, 1//3) │                          0 │        1.86279 │
│       (3, 2//3) │     -0.0354767-0.0658352im │       0.570874 │
│          (3, 1) │                          0 │        1.54481 │
│          (4, 0) │   -0.000624119-0.0026475im │       0.549786 │
│       (4, 1//4) │                          0 │        1.38073 │


## r = 4

In [22]:
V1 = Field(c, r=4, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=3//2, t=1, u=3//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (3//2, 0) │                  1.0+0.0im │            0.0 │
│ (3//2, 2//3) │      0.0342662+0.0794806im │        1.09597 │
│    (5//2, 0) │         -44322.7+3353.93im │        6.00573 │
│ (5//2, 2//5) │          29384.9+4250.18im │        4.38124 │
│ (5//2, 4//5) │         -9676.43+4213.61im │        5.41291 │
│    (7//2, 0) │  1.01538e+11+5.70352e+11im │        6.42639 │
│ (7//2, 2//7) │ -1.04149e+11-4.71809e+11im │        5.61217 │
│ (7//2, 4//7) │   5.90382e+10+3.3651e+11im │        6.34143 │
│ (7//2, 6//7) │ -2.76353e+10-8.90587e+10im │        4.48183 │
└──────────────┴────────────────────────────┴────────────────┘
Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────

In [23]:
V1 = Field(c, r=4, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1, t=1, u=2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -1.61672-0.335115im │      0.0980597 │
│    (2, 0) │          7770.31+6365.37im │       0.516735 │
│ (2, 1//2) │          8508.64+32.5809im │       0.492035 │
│    (2, 1) │         -2240.98+133.896im │       0.705569 │
│    (3, 0) │ -4.07303e+09+1.09083e+10im │       0.574106 │
│ (3, 1//3) │  8.01291e+09-1.01973e+10im │       0.493071 │
│ (3, 2//3) │ -4.45178e+09+9.31074e+09im │       0.565767 │
│    (3, 1) │  6.25392e+09-7.58059e+09im │       0.490338 │
│    (4, 0) │  2.15571e+19-1.04552e+19im │       0.379423 │
│ (4, 1//4) │ -2.12867e+19+1.02813e+19im │       0.379644 │
│ (4, 1//2) │  2.05361e+19-1.01239e+19im │       0.379941 │
│ (4, 3//4) │ -1.92927e+19+9.71093e+18im │       0.380444 │
│    (4, 1) │  1.76627e+19-9.3

In [24]:
V1 = Field(c, r=4, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=3//2, t=1, u=2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
show(stdout, sys.structure_constants, rmax=4)

Channel s
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (3//2, 0) │                  1.0+0.0im │            0.0 │
│ (3//2, 2//3) │         1.37369-0.158628im │       0.617097 │
│    (5//2, 0) │          -54466.8-2514.9im │       0.912077 │
│ (5//2, 2//5) │          2615.28-886.588im │       0.990409 │
│ (5//2, 4//5) │          3858.16+14488.0im │       0.420597 │
│    (7//2, 0) │  1.84116e+11+9.98592e+11im │       0.859899 │
│ (7//2, 2//7) │ -2.62974e+11-8.13801e+11im │       0.829991 │
│ (7//2, 4//7) │  1.10089e+11+5.87316e+11im │       0.859378 │
│ (7//2, 6//7) │ -9.12422e+10-1.42122e+11im │       0.769407 │
└──────────────┴────────────────────────────┴────────────────┘
Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────

# Sphere four-point

In [8]:
c_s = CentralCharge(β = c.β / sqrt(big(2)))
V1_s = Field(c_s, r=3, s=0)
V012 = Field(c_s, r=0, s=1//2, diagonal=true)
co_s = Correlation(V012, V1_s, V012, V012, Δmax=Δmax)
SOn_s = LoopSpectrum(c_s, co_s.Nmax)
sig_s = (s=2, t=2, u=2)
chan_spectra_s = ChannelSpectra(
    co_s, SOn_s, sig_s;
    interchiral=true, Δmax=Δmax, parity=1
)
sys_s = BootstrapSystem(chan_spectra_s)

evaluate_blocks!(sys_s)
compute_linear_system!(sys_s)
solve!(sys_s)
show(stdout, sys_s.structure_constants, rmax=4)

Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (2, 0) │                  1.0+0.0im │            0.0 │
│ (2, 1//2) │                          0 │       0.735103 │
│    (2, 1) │      -0.191342+0.0855238im │    8.16477e-15 │
│    (3, 0) │                          0 │       0.766744 │
│ (3, 1//3) │                          0 │       0.808172 │
│ (3, 2//3) │                          0 │       0.447577 │
│    (3, 1) │                          0 │       0.763083 │
│    (4, 0) │  -4.85717e-05+0.00070656im │    9.93839e-12 │
│ (4, 1//4) │                          0 │       0.286287 │
│ (4, 1//2) │ -2.94735e-05-0.000592564im │    4.75552e-12 │
│ (4, 3//4) │                          0 │         0.4432 │
│    (4, 1) │  3.77928e-05+6.72663e-05im │    7.34695e-12 │
└───────────┴────────────────────────────┴────────────────┘
Channel t
┌───────────┬───────